# 04 Hotel Database
## Assignment

Design, populate, and query a database for a hotel reservation system with the following business rules:

1. The hotel has a number of rooms of two types: Deluxe and Suite
2. For every night, some rooms are made available for reservation for a specific price.
3. A guest can make a reservation for an avavilable room for one night. The reservation must include credit card payment info. At most one reservation can be made per night per room.
4. A guest can check into a room that has been reserved. An attempt to check in without a reservation will generate an error.
5. A guest can check out only after checking in. An attempt to check out multiple times or check out without checking in will generate an error.

Your Python code should provide the following:

1. A section to create the tables. The design must be in 3rd normal form following the conventions discussed in class and enforcing the business rules above.

2. Provide code to populate rooms and room availability with prices.

3. The function `reserve_room(room, date, guest_name, credit_card)` to make a reservation. A script that populates at least 300 reservations (e.g. use `faker`)

4. The functions `checkin(room, date)` and `checkout(room, date)` to check guests in and out. Write a script that invokes `checkin` and `checkout` for a buncha guests.  Demonstrate that that the functions enforces the rules of the business.

5. Write a query to list all guests who have stayed in a given room in 2021.

6. Write a query to list all dates on which a specific guest stayed at the hotel.



# Define the database

In [1]:
import datajoint as dj
schema = dj.Schema('hotel')

[2024-09-03 17:12:35,012][INFO]: Connecting root@localhost:3306
[2024-09-03 17:12:35,032][INFO]: Connected root@localhost:3306


In [2]:
@schema
class Room(dj.Manual):
    definition = """
    room : int
    --- 
    room_type : enum('Deluxe', 'Suite')
    """

In [3]:
@schema
class RoomAvailable(dj.Manual):
    definition = """
    -> Room
    date : date 
    ---
    price : decimal(6, 2)
    """

In [4]:
@schema
class Guest(dj.Manual):
    definition = """
    guest_id : int unsigned
    --- 
    guest_name : varchar(60)
    index(guest_name)
    """

In [5]:
@schema
class Reservation(dj.Manual):
    definition = """
    -> RoomAvailable
    ---
    -> Guest
    credit_card : varchar(80)
    """

In [6]:
@schema
class CheckIn(dj.Manual):
    definition = """
    -> Reservation
    """

In [7]:
@schema
class CheckOut(dj.Manual):
    definition = """
    -> CheckIn
    """

# Populate Room Availability

In [12]:
import faker
import random
import datetime
import tqdm
fake = faker.Faker()

In [10]:
# populate rooms
Room.insert((i, "Deluxe" if i%2 else "Suite") for i in range(80))

In [11]:
# Populate Room availability: 45 days starting on start_date
start_date = datetime.date(2023, 11, 1)
days = 45

for day in tqdm.tqdm(range(days)):
    price = random.randint(40, 350) 
    RoomAvailable.insert(
        dict(key, 
             date=start_date + datetime.timedelta(days=day), 
             price=price) for key in Room.fetch('KEY'))
    

100%|██████████| 45/45 [00:00<00:00, 134.28it/s]


# Functions 

In [13]:
class HotelException(Exception): pass
class RoomUnavailable(HotelException): pass
class RoomAlreadyReserved(HotelException): pass
class AlreadyChecked(HotelException): pass
class NoReservation(HotelException): pass
class NotCheckedIn(HotelException): pass

In [14]:
def reserve_room(room, date, guest_name, credit_card):
    # lookup guest by name
    keys = (Guest & {'guest_name': guest_name}).fetch('KEY')
    
    if keys:
        # if multiple found, use the first, for example
        key = keys[0]  
    else:
        # if not registered before, create a new guest id
        key = dict(guest_id=random.randint(0, 2**32-1))
        Guest.insert1(dict(key, guest_name=guest_name))
    
    try:
        Reservation.insert1(
            dict(key, room=room, date=date, credit_card=credit_card))
    except dj.errors.DuplicateError:
        raise RoomAlreadyReserved(room, date.isoformat()) from None
    except dj.errors.IntegrityError:
        raise RoomUnavailable(room, date.isoformat()) from None

In [15]:
def check_in(room, date):
    try:
        CheckIn.insert1(dict(room=room, date=date))
    except dj.errors.DuplicateError:
        raise AlreadyChecked(room, date.isoformat()) from None
    except dj.errors.IntegrityError:
        raise NoReservation(room, date.isoformat()) from None

In [16]:
def check_out(room, date):
    try:
        CheckOut.insert1(dict(room=room, date=date))
    except dj.errors.DuplicateError:
        raise AlreadyChecked(room, date.isoformat()) from None
    except dj.errors.IntegrityError:
        raise NotCheckedIn(room, date.isoformat()) from None

# Operations

In [17]:
# make a bunch of random reservations

number_of_guests = 100
max_nights = 20

for i in tqdm.tqdm(range(number_of_guests)):
    guest = fake.name()
    credit_card=' '.join((fake.credit_card_number(), 
                          fake.credit_card_expire(), 
                          fake.credit_card_security_code()))
    
    for j in range(random.randint(1, max_nights)):
        date =  fake.date_between_dates(start_date, start_date+datetime.timedelta(days=45))
        room = random.randint(0, 80)
        try:
            reserve_room(room, date, guest, credit_card)
        except HotelException as e:
            print(repr(e))
            
# show successful reservations
Reservation()

  2%|▏         | 2/100 [00:00<00:04, 19.65it/s]

  4%|▍         | 4/100 [00:00<00:06, 13.73it/s]

RoomUnavailable(80, '2023-11-13')
RoomUnavailable(80, '2023-12-02')


  9%|▉         | 9/100 [00:00<00:05, 16.64it/s]

RoomAlreadyReserved(36, '2023-12-09')
RoomUnavailable(80, '2023-11-14')
RoomAlreadyReserved(65, '2023-11-08')
RoomAlreadyReserved(4, '2023-11-17')
RoomAlreadyReserved(72, '2023-11-09')


...

 96%|█████████▌| 96/100 [00:04<00:00, 27.31it/s]

RoomAlreadyReserved(42, '2023-12-14')
RoomAlreadyReserved(19, '2023-11-22')
RoomAlreadyReserved(7, '2023-11-02')
RoomAlreadyReserved(70, '2023-11-20')
RoomAlreadyReserved(76, '2023-12-03')
RoomAlreadyReserved(77, '2023-12-09')
RoomAlreadyReserved(58, '2023-11-08')
RoomAlreadyReserved(51, '2023-11-16')
RoomAlreadyReserved(47, '2023-11-19')
RoomAlreadyReserved(5, '2023-12-14')
RoomAlreadyReserved(10, '2023-11-05')
RoomAlreadyReserved(78, '2023-12-04')
RoomAlreadyReserved(63, '2023-11-02')
RoomAlreadyReserved(79, '2023-11-29')
RoomAlreadyReserved(28, '2023-12-15')
RoomAlreadyReserved(26, '2023-11-08')
RoomAlreadyReserved(69, '2023-11-05')
RoomAlreadyReserved(59, '2023-12-09')
RoomAlreadyReserved(71, '2023-12-02')


100%|██████████| 100/100 [00:04<00:00, 22.18it/s]

RoomAlreadyReserved(27, '2023-11-10')
RoomAlreadyReserved(43, '2023-12-10')
RoomAlreadyReserved(40, '2023-11-28')
RoomAlreadyReserved(32, '2023-11-06')
RoomAlreadyReserved(75, '2023-11-06')
RoomAlreadyReserved(38, '2023-12-06')
RoomAlreadyReserved(42, '2023-12-01')
RoomAlreadyReserved(61, '2023-12-09')


room,date,guest_id,credit_card
0,2023-11-01,2181298725,370847074797867 11/28 252
0,2023-11-09,2134517628,4206960338250577690 10/31 4171
0,2023-11-11,1245454711,377809788669900 02/33 8750
0,2023-11-14,2522954717,4674030165763 03/34 534
0,2023-11-18,4200689128,6011438170332221 06/33 0167
0,2023-11-23,3515172225,6551377824190880 06/25 555
0,2023-11-26,456912289,3535348020023777 12/33 326
0,2023-12-01,1998827306,3547998829075687 05/34 551
0,2023-12-03,2153171054,3571568485218472 09/31 445
0,2023-12-04,562506096,4606721665831850290 07/28 671


In [18]:
Guest()

guest_id,guest_name
3990092660,Abigail Green
1901131669,Adam Brown
1450344758,Adam Miller
409512713,Adam Norman
3577012781,Amanda Monroe
3515172225,Angela Adams
3154279274,Angela Gonzales
1958355880,Angela Valenzuela
2181298725,Anna Gomez
321437229,Brad Lowe


In [19]:
# Try check in
check_in(2, datetime.date(2023, 11, 2))

NoReservation: (2, '2023-11-02')

In [20]:
# checkin a bunch of people
checkins = random.sample(Reservation().fetch('KEY'), k=int(0.9*len(Reservation())))
for r in tqdm.tqdm(checkins):
    try: 
        check_in(**r)
    except AlreadyChecked as e:
        print(repr(e))


  0%|          | 0/828 [00:00<?, ?it/s]

100%|██████████| 828/828 [00:02<00:00, 330.90it/s]


In [21]:
CheckIn()

room,date
0,2023-11-09
0,2023-11-11
0,2023-11-14
0,2023-11-18
0,2023-11-23
0,2023-11-26
0,2023-12-01
0,2023-12-03
0,2023-12-05
0,2023-12-06


In [22]:
# Try duplicate checkin
check_in(**checkins[0])

AlreadyChecked: (46, '2023-11-08')

In [23]:
# Try checkout

check_out(2, datetime.date(2023, 10, 2))

NotCheckedIn: (2, '2023-10-02')

In [24]:
# checkout a bunch of people
checkouts = random.sample(CheckIn().fetch('KEY'), k=int(0.9*len(CheckIn())))
for r in tqdm.tqdm(checkouts):
    try:
        check_out(**r)
    except AlreadyChecked as e:
        print(repr(e))

  4%|▍         | 32/745 [00:00<00:02, 317.79it/s]

100%|██████████| 745/745 [00:02<00:00, 356.68it/s]


In [25]:
# try duplicate checkout

check_out(**checkouts[0])

AlreadyChecked: (19, '2023-11-03')